# Data Modelling

## Get the data from the previous notebook

In [ ]:
import json
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

USE_MERGED_DATA = True

#Get the data from the previous notebook, as csv files, and load them into dataframes
awards_players_df = pd.read_csv('../prep_data/awards_players_df.csv')
coaches_df = pd.read_csv('../prep_data/coaches_df.csv')
players_df = pd.read_csv('../prep_data/players_df.csv')
players_teams_df = pd.read_csv('../prep_data/players_teams_df.csv')
series_post_df = pd.read_csv('../prep_data/series_post_df.csv')
# teams_df = pd.read_csv('../prep_data/teams_df_with_average_attributes_of_players.csv.csv') isto dá 100% nalgumas cenas, mas muito menos no rolling window, deve ser um overfit manhoso, estranho
# a soma dos 3 melhores de cada atributo dá essencialmente o mesmo que a média e normalizar o teams_df original não fez diferença nenhuma
if USE_MERGED_DATA:
    teams_df = pd.read_csv('../prep_data/teams_df_with_sum_of_3_best_attributes_of_players.csv')
else:
    teams_df = pd.read_csv('../prep_data/teams_df.csv')
teams_post_df = pd.read_csv('../prep_data/teams_post_df.csv')

#make a dictionary with all the dataframes
dfs = {'awards_players_df': awards_players_df, 'coaches_df': coaches_df, 'players_df': players_df, 'players_teams_df': players_teams_df, 'series_post_df': series_post_df, 'teams_df': teams_df, 'teams_post_df': teams_post_df}


In [ ]:
#Print all the dataframes
for key in dfs:
    print(key)
    print(dfs[key].head())
    print('\n')

# MODELING

In [ ]:
def print_confusion_matrix(cm, target_names):
    df_cm = pd.DataFrame(cm, index=target_names, columns=target_names)
    sns.heatmap(df_cm, annot=True, cmap='Blues', fmt='g')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

## Decision Tree

Run a simple model on the teams table to predict if a team will make it to the playoffs or not. Let's see how it behaves.

In [ ]:
# teams_df

# split into train years and test year (year 10)
teams_train_df = teams_df[teams_df['year'] < 10]
teams_test_df = teams_df[teams_df['year'] == 10]

In [ ]:
#see all null values in teams_df
teams_df.isnull().sum()

In [ ]:
# run a tree decision classifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
from sklearn.ensemble import RandomForestClassifier


# Split the data into features and target label
target_names = ['No Playoffs', 'Playoffs']

y_train = teams_train_df['playoff']
y_test = teams_test_df['playoff']
X_train = teams_train_df.drop(['playoff', 'year', 'tmID', 'firstRound', 'semis', 'finals'], axis=1)
X_test = teams_test_df.drop(['playoff', 'year', 'tmID', 'firstRound', 'semis', 'finals'], axis=1)

if USE_MERGED_DATA:
    X_train = X_train.drop(['W_post', 'L_post'], axis=1)
    X_test = X_test.drop(['W_post', 'L_post'], axis=1)

clf = DecisionTreeClassifier()

# Fit the classifier on the training data
clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Create a confusion matrix to evaluate the model and print with labels
cm = confusion_matrix(y_test, y_pred)
print_confusion_matrix(cm, target_names)

In [ ]:
top_teams_indices = y_pred.argsort()[-8:][::-1]  # Get the indices of the top 8 teams

# Convert the labels to numeric format (0 for 'No Playoff', 1 for 'Playoff')
y_pred_numeric = [1 if i in top_teams_indices else 0 for i in range(len(y_pred))]

# Calculate the accuracy of the modified predictions
accuracy = accuracy_score(y_test, y_pred_numeric)
print("Modified Accuracy (Top 8 Teams as Playoff):", accuracy)

# Create a confusion matrix to evaluate the model and print with labels
cm = confusion_matrix(y_test, y_pred_numeric)
print_confusion_matrix(cm, target_names)

## Rolling window

We will use a rolling window to predict the next season. We will use the last 3 seasons to predict the next one.

In [ ]:
# Define the rolling window size (number of past years to consider)
ROLLING_WIN_SIZE = 3  # You can adjust this value
PREDICTION_YEAR = 10

historical_data_df = teams_df[teams_df['year'] < PREDICTION_YEAR]

def create_rolling_window_dataset(df, window_size):
    rolling_window_data = []
    for tmID, group in df.groupby('tmID'):
        group = group.sort_values(by='year')
        for i in range(len(group) - window_size + 1):
            window = group.iloc[i:i + window_size]
            rolling_window_data.append(window)
    return rolling_window_data

In [ ]:
# Split the data into features and target label
target_names = ['No Playoff', 'Playoff']

# Create the rolling window datasets
rolling_window_historic = create_rolling_window_dataset(historical_data_df,  ROLLING_WIN_SIZE)

# Define weights (you can adjust these weights as needed)
weights = [0.2, 0.3, 0.5]  # Assign higher weight to the last year

#Columns to apply the weighted moving average to
# weighted_features = ['o_fta', 'o_3pm', 'o_oreb', 'o_asts', 'o_stl', 'o_to', 'o_blk', 'd_3pa', 'd_oreb', 'd_dreb', 'd_asts', 'd_stl', 'd_to', 'd_blk', 'attend']

weighted_features = ['o_fta', 'o_3pm', 'o_oreb', 'o_asts', 'o_stl', 'o_to', 'o_blk', 'd_3pa', 'd_oreb', 'd_dreb', 'd_asts', 'd_stl', 'd_to', 'd_blk', 'attend']

if USE_MERGED_DATA:
    weighted_features += ['num_seasons', 'height', 'weight', 'stint', 'GP', 'threeAttempted', 'dq', 'PostoRebounds', 'PostBlocks', 'PostthreeAttempted', 'PostDQ', 'FG%', 'FT%', 'Avg_Assists_Per_Game', 'Avg_Blocks_Per_Game', 'Avg_Steals_Per_Game', 'Points_Ratio', 'POSITION_METRIC']

# Create a new dataframe to store the rolling window data
rolling_data = pd.DataFrame(columns=teams_df.columns)

for window in rolling_window_historic:
    #For each window, we will add a line to the rolling_data dataframe, as follows:
    #1. Create a new empty line for the rolling_data dataframe
    rolling_data_line = pd.DataFrame(columns=teams_df.columns)

    #2. Get the weighted average of the weighted features
    #Note: The weighted average is calculated as follows:
    #weighted average = sum of (weight * feature value) / sum of weights
    #For example, if the weights are [0.2, 0.3, 0.5] and the feature values are [10, 20, 30], then the weighted average is:
    #weighted average = (0.2 * 10 + 0.3 * 20 + 0.5 * 30) / (0.2 + 0.3 + 0.5) = 24
    for feature in weighted_features:
        weighted_average = np.average(window[feature].values, weights=weights)
        rolling_data_line[feature] = [weighted_average]

    #3. Join that data with the data from the following year (the year we want to predict)
    #Note: The data from the following year will be used as the target label, get next year from the teams_df dataframe
    next_year = window['year'].max() + 1
    next_year_data = teams_df[(teams_df['tmID'] == window['tmID'].values[0]) & (teams_df['year'] == next_year)]

    #Add year, confID, tmID, and playoff columns
    try:
        rolling_data_line['year'] = next_year
        rolling_data_line['tmID'] = window['tmID'].values[0]
        rolling_data_line['confID'] = next_year_data['confID'].values[0]
        rolling_data_line['playoff'] = next_year_data['playoff'].values[0]
    except:
        #If there is no data for the next year, skip this line
        #print('No data for next year')
        continue
    
    #4. Concat the data to the rolling_data dataframe
    rolling_data = pd.concat([rolling_data, rolling_data_line])

rolling_data = rolling_data.reset_index(drop=True)

# Split the data into features and target label
# Test data will be the 10th year
target_names = ['No Playoff', 'Playoff']

y_train = rolling_data[rolling_data['year'] < 10]['playoff'].astype(int)
y_test = rolling_data[rolling_data['year'] == 10]['playoff'].astype(int)
X_train = rolling_data[rolling_data['year'] < 10].drop(['playoff', 'year', 'tmID', 'firstRound', 'semis', 'finals'], axis=1)
X_test = rolling_data[rolling_data['year'] == 10].drop(['playoff', 'year', 'tmID', 'firstRound', 'semis', 'finals'], axis=1)

if USE_MERGED_DATA:
    X_train = X_train.drop(['W_post', 'L_post'], axis=1)
    X_test = X_test.drop(['W_post', 'L_post'], axis=1)

clf = RandomForestClassifier(n_estimators=100)
# clf = DecisionTreeClassifier()

# Fit the classifier on the training data
clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Create a confusion matrix to evaluate the model and print with labels
cm = confusion_matrix(y_test, y_pred)
print_confusion_matrix(cm, target_names)


In [ ]:
top_teams_indices = y_pred.argsort()[-8:][::-1]  # Get the indices of the top 8 teams

# Convert the labels to numeric format (0 for 'No Playoff', 1 for 'Playoff')
y_pred_numeric = [1 if i in top_teams_indices else 0 for i in range(len(y_pred))]

# Calculate the accuracy of the modified predictions
accuracy = accuracy_score(y_test, y_pred_numeric)
print("Modified Accuracy (Top 8 Teams as Playoff):", accuracy)

# Create a confusion matrix to evaluate the model and print with labels
cm = confusion_matrix(y_test, y_pred_numeric)
print_confusion_matrix(cm, target_names)

## Random Forest

In [ ]:
# Split the data into features and target label

target_names = ['No Playoff', 'Playoff']

y_train = teams_train_df['playoff']
y_test = teams_test_df['playoff']
X_train = teams_train_df.drop(['playoff', 'year', 'tmID', 'firstRound', 'semis', 'finals'], axis=1)
X_test = teams_test_df.drop(['playoff', 'year', 'tmID', 'firstRound', 'semis', 'finals'], axis=1)

if USE_MERGED_DATA:
    X_train = X_train.drop(['W_post', 'L_post'], axis=1)
    X_test = X_test.drop(['W_post', 'L_post'], axis=1)

random_forest_classifier = RandomForestClassifier(n_estimators=100)

# Fit the classifier on the training data
random_forest_classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred = random_forest_classifier.predict(X_test)

feature_importances = pd.DataFrame(random_forest_classifier.feature_importances_, index = X_train.columns, columns=['importance']).sort_values('importance', ascending=False)
display(feature_importances)

# print the used Decision Tree
plt.figure(figsize=(20,20))
plot_tree(random_forest_classifier.estimators_[0], feature_names=X_train.columns, filled=True)
plt.show()

In [ ]:


# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

# Create a confusion matrix to evaluate the model and print with labels
cm = confusion_matrix(y_test, y_pred)
print_confusion_matrix(cm, target_names)


In [ ]:
# try knn
from sklearn.neighbors import KNeighborsClassifier

knn_classifier = KNeighborsClassifier(n_neighbors=5)

# Fit the classifier on the training data
knn_classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred = knn_classifier.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)


In [ ]:

# Fiz isto aqui porque não encontrei onde foram criados os csvs de /prep_data e queria garantir que estava usar a mesma coisa
# SPOILER: fez 0 diferença
"""
for col in dfs['teams_df'].select_dtypes(include=np.number).columns:
    if col not in ['year', 'confID', 'playoff', 'firstRound', 'semis', 'finals']:
        aux = (dfs['teams_df'][col] - dfs['teams_df'][col].min()) / (dfs['teams_df'][col].max() - dfs['teams_df'][col].min())
        dfs['teams_df'][col] = aux.round(3)
# store in csv in prep_data with 2 appended to the name
dfs['teams_df'].to_csv('../prep_data/teams_df2.csv', index=False)
"""